# Hidden Markov Model project
## Machine Learning Fundamentals

|First name|Last name|Master program|Contribution|
|----------|---------|--------------|-------------|
|Aloys|Lana|DSC|33%|
|Arthur|Micol|DSC|33%|
|Florian|Doffemont|DSC|33%|


We use https://colab.research.google.com/. It have an issue in the imports at the beginning (WARNING : you need version 0.2.6 of hmmlearn)

To fix it : 
- Run the 1st code import cell
- You have an error
- Click at the top on "execution" -> "Redémarer l'environnement d'éxécution"
- Run the cell again
- Its done !

# WARNING : you need version 0.2.6 of hmmlearn

In [1]:
!pip uninstall -y numpy scikit-learn joblib hmmlearn
!pip install hmmlearn==0.2.6
import numpy as np
import hmmlearn
from hmmlearn import hmm

Found existing installation: numpy 1.24.2
Uninstalling numpy-1.24.2:
  Successfully uninstalled numpy-1.24.2
Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: joblib 1.2.0
Uninstalling joblib-1.2.0:
  Successfully uninstalled joblib-1.2.0
Found existing installation: hmmlearn 0.2.6
Uninstalling hmmlearn-0.2.6:
  Successfully uninstalled hmmlearn-0.2.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached hmmlearn-0.2.6-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (369 kB)
  Using cached numpy-1.24.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached scikit_learn-1.2.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are inst

# WARNING : check that the hmmlearn installed is version 0.2.6

In [2]:
!pip show hmmlearn  # check that the version installed is 0.2.6

Name: hmmlearn
Version: 0.2.6
Summary: Hidden Markov Models in Python with scikit-learn like API
Home-page: https://github.com/hmmlearn/hmmlearn
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.9/dist-packages
Requires: numpy, scikit-learn, scipy
Required-by: 


# imports

In [3]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from tqdm import tqdm
from matplotlib import pyplot as plt  # show graph

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, \
    f1_score, roc_auc_score

## In this notebook we will look at the NER dataset and use it to understand HMM and also construct a POS tagger at the same time.

### Data Description:
#### sentence: this column donates to which sentence the word belongs
#### Word: the word in the sentence
#### POS: Associated POS tag for the word

## Load the data

If you imported the dataset "NER dataset.csv" to you google drive, you can use the following to mount and import it.

In [4]:
# Loading the data
from google.colab import drive

drive.mount('/content/drive')

data = pd.read_csv("/content/drive/MyDrive/MLF_project/data/NER dataset.csv", encoding='latin1')

Mounted at /content/drive


Otherwise, if you are working locally or if you just uploaded the dataset for this session on the drive, you can use the following to import it.

In [ ]:
# Loading the data
# data = pd.read_csv("./data/NER dataset.csv", encoding='latin1')

In [5]:
data = data.fillna(method="ffill")
data = data.rename(columns={'Sentence #': 'sentence'})
data.head(5)

,sentence,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


# Data pre-processing
If you want to do some pre-processing (lowercase any words, remove stop words, replace numbers/names by a unique NUM/NAME token, etc.) you can do it here in the pipeline.

Note : you could create a new dataset `data_pre_precessed = pre_process(data)` to keep both version and compare the effect of you pre-processing.

In [6]:
# Define a function `pre_processing` that takes a dataframe `data` and applies the `lower()` method to the "Word" column of the dataframe, which converts all words to lowercase. 
# Then, it returns the dataframe with the lowercase words.
def pre_processing(data):
    data['Word'] = data['Word'].apply(lambda x: x.lower())
    return data

# Call the `pre_processing` function with the argument `data`, which should be a dataframe containing words in a column called "Word". 
# The modified dataframe is stored in a new variable `dataPreProcess`, and the first five rows of this dataframe are displayed using the `head()` method.
dataPreProcess = pre_processing(data)
dataPreProcess.head(5)


,sentence,Word,POS,Tag
0,Sentence: 1,thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


First let's collect the unique words and the unique POS tags in the dataset, we will use this to construct the HMM later

In [7]:
tags = list(set(dataPreProcess.POS.values))  # Unique POS tags in the dataset
words = list(set(dataPreProcess.Word.values))  # Unique words in the dataset
len(tags), len(words)

(42, 31817)

### We have 42 different tags and 35,178 different words, so the HMM that we construct will have the following properties
- The hidden states of the this HMM will correspond to the POS tags, so we will have 42 hidden states.
- The Observations for this HMM will correspond to the sentences and their words.

#### Before constructing the HMM, we will split the data into train and test.

In [8]:
# Splitting the data into input features and target variable
y = dataPreProcess.POS # Target variable (labels)
X = dataPreProcess.drop('POS', axis=1) # Input features (datas)

# Using GroupShuffleSplit to split the data into train and test sets
gs = GroupShuffleSplit(n_splits=2, test_size=.33, random_state=42) # Initialize GroupShuffleSplit object : Split by 2 with 33% test and 0.66% train
train_ix, test_ix = next(gs.split(X, y, groups=dataPreProcess['sentence'])) # Generate indices for train and test sets

# Creating train and test sets using the generated indices
data_train = dataPreProcess.loc[train_ix] # Train set 
data_test = dataPreProcess.loc[test_ix] # Test set

In [9]:
# Showing the first 5 rows of the train set
data_train.head(5)

,sentence,Word,POS,Tag
24,Sentence: 2,families,NNS,O
25,Sentence: 2,of,IN,O
26,Sentence: 2,soldiers,NNS,O
27,Sentence: 2,killed,VBN,O
28,Sentence: 2,in,IN,O


In [10]:
# Showing the first 5 rows of the test set
data_test.head(5)

,sentence,Word,POS,Tag
0,Sentence: 1,thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


Now lets encode the POS and Words to be used to generate the HMM.

In [11]:
# Select a random sample of 15% of the training data
dfupdate = data_train.sample(frac=.15, replace=False, random_state=42)

# Replace the words in the selected sample with 'UNKNOWN'
dfupdate.Word = 'UNKNOWN'

# Update the training data with the modified sample
data_train.update(dfupdate)

# Collect the unique words in the modified training data
words = list(set(data_train.Word.values))

# Convert the unique words and tags into numerical IDs
word2id = {w: i for i, w in enumerate(words)}
tag2id = {t: i for i, t in enumerate(tags)}
id2tag = {i: t for i, t in enumerate(tags)}

# Get the number of unique tags and words in the dataset
len(tags), len(words)

(42, 25100)

In your theory classes you might have seen that the Hidden Markov Models can be learned by using the Baum-Welch algorithm by just using the observations.
Although we can learn the Hidden States (POS tags) using Baum-Welch algorithm,We cannot map them back the states (words) to the POS tag. So for this exercise we will skip using the BW algorithm and directly create the HMM.

For creating the HMM we should build the following three parameters. 
- `startprob_`
- `transmat_`
- `emissionprob_`

To construct the above mentioned paramters let's first create some useful matrices that will assist us in creating the above three parameters

In [12]:
# Compute the count of POS tags in the dataset
count_tags = dict(data_train.POS.value_counts())

# Compute the count of each word associated with each POS tag
count_tags_to_words = data_train.groupby(['POS']).apply(
    lambda grp: grp.groupby('Word')['POS'].count().to_dict()).to_dict()

# Compute the count of each POS tag at the beginning of a sentence
count_init_tags = dict(data_train.groupby('sentence').first().POS.value_counts())

# Create a matrix to store the frequency of transitions between POS tags
count_tags_to_next_tags = np.zeros((len(tags), len(tags)), dtype=int)

# Loop over the sentences and update the matrix with transition frequencies
sentences = list(data_train.sentence)
pos = list(data_train.POS)
for i in tqdm(range(len(sentences)), position=0, leave=True):
    # Check if the current sentence is not the first in the dataset
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        # Get the previous and current POS tags
        prevtagid = tag2id[pos[i - 1]]
        nexttagid = tag2id[pos[i]]
        # Update the matrix with the transition frequency
        count_tags_to_next_tags[prevtagid][nexttagid] += 1

100%|██████████| 702936/702936 [00:00<00:00, 830181.48it/s]


Now Let's build the parameter matrices 

In [13]:
# Initialize the parameter matrices
startprob = np.zeros((len(tags),))      # Starting probability matrix for each tag with 0
transmat = np.zeros((len(tags), len(tags)))   # Transition probability matrix for each tag with 0
emissionprob = np.zeros((len(tags), len(words)))  # Emission probability matrix for each tag and word with 0

# Calculate the total number of sentences
num_sentences = sum(count_init_tags.values())

# Calculate the sum of counts of transitions for each tag to the next tags
sum_tags_to_next_tags = np.sum(count_tags_to_next_tags, axis=1)

# Iterate over each tag and its corresponding tag id
for tag, tagid in tqdm(tag2id.items(), position=0, leave=True):
    
    # Get the count of the tag
    floatCountTag = float(count_tags.get(tag, 0))
    
    # Calculate the starting probability of the tag
    startprob[tagid] = count_init_tags.get(tag, 0) / num_sentences
    
    # Calculate the emission probability of the tag for each word
    for word, wordid in word2id.items():
        emissionprob[tagid][wordid] = count_tags_to_words.get(tag, {}).get(word, 0) / floatCountTag
    
    # Calculate the transition probability of the tag to each of the other tags
    for tag2, tagid2 in tag2id.items():
        transmat[tagid][tagid2] = count_tags_to_next_tags[tagid][tagid2] / sum_tags_to_next_tags[tagid]

100%|██████████| 42/42 [00:00<00:00, 58.69it/s]


## Task 1: Similar to how we built the hidden state transition probability matrix as shown above, you will built the transition probability between the words. With this matrix write a function that can calculate the log likelihood given a sentence.

In [14]:
# We import that because we had the NameError: name 'List' is not defined
from typing import List

# Create the word transition matrix
word_transition_matrix = np.zeros((len(words), len(words)), dtype=np.float64)

# Iterate over the sentences in the dataset
for sentence_id in tqdm(data_train['sentence'].unique(), position=0, leave=True):
    # Get the words of the sentence
    sentence_words = data_train.loc[data_train['sentence'] == sentence_id, 'Word'].values
    
    # Iterate over the words of the sentence, except for the last word
    for i in range(len(sentence_words) - 1):
        # Get the ID of the current and next words
        current_word_id = word2id[sentence_words[i]]
        next_word_id = word2id[sentence_words[i + 1]]
        
        # Increment the count of the transition from the current to the next word
        word_transition_matrix[current_word_id, next_word_id] += 1

# Normalize each row of the word_transition_matrix
for i in range(len(words)):
    row_sum = np.sum(word_transition_matrix[i])
    if row_sum > 0:
        word_transition_matrix[i] /= row_sum

# print(word_transition_matrix)


100%|██████████| 32132/32132 [23:33<00:00, 22.73it/s]


#### The log-likelihood function 

In [15]:
def calculate_log_likelihood(sentence: List[str], word_transition_matrix) -> float:
    """
    Calculate the log-likelihood of a given sentence using the word transition matrix.
    
    Args:
        sentence (List[str]): A list of words representing the sentence.
        word_transition_matrix : A word transition matrix.
    
    Returns:
        float: The log-likelihood of the sentence.
    """
    log_likelihood = 0
    
    # Iterate through the words in the sentence and calculate the log-likelihood
    for i in range(len(sentence) - 1):

        # Get the current word and next word from the sentence
        current_word = sentence[i]
        next_word = sentence[i + 1]

        # Check if the current and next words are in word2id, otherwise use 'UNKNOWN'
        current_word_id = word2id.get(current_word, word2id['UNKNOWN'])
        next_word_id = word2id.get(next_word, word2id['UNKNOWN'])
        
        # Get the transition probability between the current and next words using their IDs
        transition_probability = word_transition_matrix[current_word_id][next_word_id]
        
        # Check if the probability is zero to avoid calculating the logarithm of zero
        if transition_probability == 0:
            log_likelihood += 0
        
        # Calculate the log-likelihood for this pair of words
        log_likelihood += np.log(transition_probability)
    
    return log_likelihood

####Now let's try this function on some exemples :


In [16]:
print(calculate_log_likelihood(["this", "is", "a", "protest", "about", "how", "the", "new", "law", "is", "not", "in", "the", "interest", "of", "the", "people"], word_transition_matrix))
print(calculate_log_likelihood(["the", "international", "conference", "will", "continue", "as", "planned", "on", "Friday"], word_transition_matrix))
print(calculate_log_likelihood(["who", "are", "you", "?"], word_transition_matrix))
print(calculate_log_likelihood(["you", "are", "not", "me"], word_transition_matrix))
print(calculate_log_likelihood(["do", "you", "expect", "to", "be", "happy", "to", "work", "late"], word_transition_matrix))

-70.96470273464142
-34.95037224672289
-15.9902178018753
-13.89888346686548
-35.11892547319604


#### Now we will continue to constructing the HMM.

We will use the hmmlearn implementation to initialize the HMM Model

In [17]:
# To check if the matrix was correctly fill
# for i in range(len(words)):
#   for y in range(len(words)):
#     if word_transition_matrix[i, y] != 0:
#       print(word_transition_matrix[i, y])

model = hmm.MultinomialHMM(n_components=len(tags), algorithm='viterbi', random_state=42)
model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob

#### Before using the HMM to predict the POS tags, we have to fix the training set as some of the words and tags in the test data might not appear in the training data so we collect this data to use it later.

In [18]:
# Replace words in data_test not in the 'words' set with 'UNKNOWN'
data_test.loc[~data_test['Word'].isin(words), 'Word'] = 'UNKNOWN'
# Convert the 'Word' column of data_test into a list
word_test = list(data_test.Word)
# Initialize an empty list for samples
samples = []
# For each word in word_test, append its word ID to samples
for i, val in enumerate(word_test):
    samples.append([word2id[val]])

# Non-pandas solution for sentence lengths
lengths = []
count = 0
sentences = list(data_test.sentence)
for i in tqdm(range(len(sentences)), position=0, leave=True):
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        count += 1
    elif i > 0:
        lengths.append(count)
        count = 1
    else:
        count = 1

# Pandas solution for sentence lengths
# Create a new column 'is_start_of_sentence' that marks the start of a new sentence
# We use pandas' 'ne' function to compare each sentence value with the previous one (using 'shift()')
# This results in a boolean series where 'True' indicates the start of a new sentence
data_test['is_start_of_sentence'] = data_test['sentence'].ne(data_test['sentence'].shift())
# Group data_test by cumulative sum of 'is_start_of_sentence'
# We use pandas' 'cumsum()' function to create a running sum of 'True' values, effectively labeling each group of sentences
# Then, we use 'groupby()' to group the data by these labels
grouped = data_test.groupby(data_test['is_start_of_sentence'].cumsum())
# Calculate the size (number of rows) of each group using pandas' 'size()' function and reset the index
sentence_lengths = grouped.size().reset_index(drop=True)
# Convert the group sizes into a list
lengthsPanda = sentence_lengths.tolist()

# Print the sentence lengths obtained using both methods
print(lengths)
print(lengthsPanda)

100%|██████████| 345639/345639 [00:00<00:00, 1910549.02it/s]


[24, 21, 28, 25, 26, 22, 34, 23, 19, 25, 22, 35, 30, 24, 26, 39, 31, 7, 34, 8, 26, 32, 14, 30, 24, 11, 17, 29, 27, 7, 32, 25, 18, 18, 12, 28, 32, 22, 21, 26, 13, 25, 33, 20, 9, 27, 36, 40, 18, 24, 11, 17, 11, 26, 14, 20, 28, 21, 41, 25, 35, 18, 21, 30, 20, 17, 18, 27, 24, 25, 26, 17, 22, 14, 22, 27, 26, 24, 21, 30, 10, 14, 32, 30, 22, 17, 16, 23, 17, 29, 23, 16, 18, 22, 23, 25, 17, 24, 29, 15, 16, 46, 14, 27, 23, 24, 22, 20, 19, 7, 12, 21, 19, 19, 40, 13, 14, 26, 18, 27, 11, 31, 33, 21, 23, 9, 19, 20, 38, 26, 23, 18, 24, 26, 21, 17, 18, 24, 26, 24, 24, 37, 34, 15, 15, 17, 19, 25, 17, 9, 41, 24, 22, 18, 12, 37, 27, 12, 21, 29, 16, 24, 29, 23, 20, 33, 17, 17, 11, 19, 43, 31, 21, 42, 13, 31, 27, 35, 31, 13, 14, 38, 26, 24, 22, 32, 29, 62, 12, 9, 30, 15, 31, 28, 29, 13, 19, 13, 12, 12, 16, 27, 23, 32, 20, 10, 23, 27, 34, 15, 13, 19, 21, 11, 26, 28, 36, 14, 15, 18, 6, 30, 9, 18, 24, 32, 29, 25, 23, 22, 21, 20, 15, 26, 17, 29, 21, 20, 30, 18, 22, 21, 25, 38, 17, 11, 34, 27, 37, 13, 26, 26, 2

Now that we have the HMM ready lets predict the best path from them.

In [19]:
# Use the 'predict' method of the 'model' object (HMM) to predict the POS tags for the 'samples' (words) 
# We also pass 'lengths' as an argument to inform the model about the sentence boundaries
pos_predict = model.predict(samples, lengths)
# Print the predicted POS tags for the given sentences
print(pos_predict)

[ 5  0  5 ... 39 41 14]


The hmmlearn predict function will give the best probable path for the given sentence using the Viterbi algorithm.

## Task 2: Using the model parameters (startprob_, transmat_, emissionprob_) write the viterbi algorithm from scratch to calculate the best probable path and compare it with the hmmlearn implementation.

Now before using these matrices 

In [20]:
def Viterbi(pi: np.array, a: np.array, b: np.array, obs: List) -> np.array:
    """
    Viterbi algorithm implementation to find the best probable path.
    
    Args:
        pi: Initial probabilities.
        a: Transition probabilities.
        b: Emission probabilities.
        obs: List of observations.
        
    Returns:
        np.array: Array of the indices of the best hidden states.
    """
    # Initialize the dynamic programming table (dp) with zeros
    # The table has N rows (number of hidden states) and T columns (number of observations)
    T = len(obs)
    N = a.shape[0]
    dp = np.zeros((N, T))

    # Initialize the backpointers table to store the most probable previous states
    backpointers = np.zeros((N, T), dtype=int)

    # Initialize the first column of the DP table by multiplying the initial probabilities (pi)
    # with the emission probabilities for the first observation (obs[0])
    dp[:, 0] = pi * b[:, obs[0]]

    # Iterate through the observations (from the second observation to the last one)
    for t in range(1, T):
        # Iterate through each hidden state
        for s in range(N):
            # For each hidden state, calculate the maximum probability by considering
            # the probability of the previous state (dp[prev_s, t-1]), the transition
            # probability from the previous state to the current state (a[prev_s, s]),
            # and the emission probability of the current observation (b[s, obs[t]])
            max_prob, max_state = max((dp[prev_s, t-1] * a[prev_s, s] * b[s, obs[t]], prev_s) for prev_s in range(N))
            
            # Store the maximum probability in the corresponding cell of the DP table (dp[s, t])
            dp[s, t] = max_prob
            
            # Store the index of the previous state that led to the maximum probability
            # in the backpointers table (backpointers[s, t])
            backpointers[s, t] = max_state

    # After filling the DP table, find the index of the final hidden state with the highest probability
    final_state = np.argmax(dp[:, -1])

    # Backtrack through the backpointers table to find the best hidden state sequence
    # Start with the final state found above and add it to the best_path list
    best_path = [final_state]

    # Iterate backward through the backpointers table, starting from the last observation (T-1)
    # and going to the first observation (1)
    for t in range(T-1, 0, -1):
        # For each observation, find the most probable previous state that led to the current state
        # (backpointers[best_path[-1], t]) and add it to the best_path list
        best_path.append(backpointers[best_path[-1], t])
        
    # The best_path list is currently in reverse order, so reverse it to get the correct order
    best_path = best_path[::-1]

    # Convert the best_path list to a numpy array and return it
    return np.array(best_path)

#### We test the viterbi algorithm and compare it with the hmmlearn implementation

In [21]:
# Prepare input data for Viterbi function
# We start by defining a sample sentence as a list of words
sample_sentence = ["You", "are", "not", "me"]

# Next, we convert each word in the sample_sentence to its corresponding index (ID) using the word2id dictionary
# If the word is not in the word2id dictionary (unknown word), we use the index for 'UNKNOWN'
sample_indices = [word2id.get(word, word2id['UNKNOWN']) for word in sample_sentence]

# Convert the sample_indices into a 2D array for compatibility with hmmlearn's predict method
sample_indices_2d = np.array([sample_indices])

# Call Viterbi function and hmmlearn implementation with the same input data and compare results
viterbi_result = Viterbi(model.startprob_, model.transmat_, model.emissionprob_, sample_indices)
hmmlearn_result = model.predict(sample_indices_2d, [len(sample_indices_2d)])



print("Custom Viterbi result:", viterbi_result)
print("hmmlearn result:", hmmlearn_result)

Custom Viterbi result: [ 5 12 25  2]
hmmlearn result: [ 5 12 25  2]


### Task 3: Let's try to form our own HMM
In this task you will try to formulate your own HMM. Image a toy example that you think that closely relates to a Hidden Markov Model.

Steps:
 1. Define your hidden states
 2. Define your observable states
 3. Randomly generate your observations

Below is an example to demonstrate:

-In this toy HMM example, we have two hidden states 'healthy' and 'sick' these states relate to the state of a pet. In this example we cannot exactly know the situation of the pet if it is 'healthy' or 'sick'

-The observable states in this formulation is the what our pet is doing, whether it is sleeping, eating or pooping. We ideally want to determine if the pet is sick or not using these observable states


```python
hidden_states = ['healthy', 'sick']
observable_states = ['sleeping', 'eating', 'pooping']
observations = []
for i in range(100):
  observations.append(random.choice(observable_states))
```

###TASK 3: Now try to formulate your HMM here.

####**Our example :**
Our HMM example is related to video games, we will attempt to determine the hidden state of a player based on their observable actions within the game

####**Hidden states :**
In this example, we have two hidden states for a  player : **"beginner"** and **"expert"**. These states represent thbe player's skill level. We cannot directly know the player's skill level, but we can try to deduce it from their actions in the game.

####**Observable states :**
The observable states in this formulation are the actions the player performs in the game, such as **"attack"**, **"defend"**, and **"use item"**. We ideally want to determine the player's skill level based on these observable states.

In [23]:
import random

# Define the hidden states representing the player's skill level
hidden_states_own_HMM = ['beginner', 'expert']

# Define the observable states representing the actions performed by the player in the game
observable_states_own_HMM = ['attack', 'defend', 'use item']

# Generate a list of 100 random observations from the observable states
observations_own_HMM = []
for i in range(100):
  observations_own_HMM.append(random.choice(observable_states_own_HMM))

print(observations_own_HMM)

['attack', 'use item', 'attack', 'defend', 'use item', 'defend', 'use item', 'defend', 'use item', 'attack', 'defend', 'use item', 'attack', 'use item', 'attack', 'attack', 'attack', 'defend', 'attack', 'use item', 'use item', 'use item', 'defend', 'attack', 'use item', 'defend', 'defend', 'attack', 'defend', 'use item', 'use item', 'defend', 'use item', 'defend', 'attack', 'attack', 'attack', 'defend', 'attack', 'use item', 'attack', 'defend', 'attack', 'use item', 'defend', 'use item', 'use item', 'defend', 'use item', 'defend', 'attack', 'use item', 'use item', 'attack', 'attack', 'defend', 'defend', 'attack', 'attack', 'attack', 'use item', 'attack', 'use item', 'attack', 'attack', 'use item', 'use item', 'defend', 'attack', 'defend', 'defend', 'attack', 'use item', 'defend', 'defend', 'attack', 'attack', 'use item', 'attack', 'attack', 'attack', 'use item', 'attack', 'use item', 'defend', 'use item', 'use item', 'attack', 'defend', 'use item', 'use item', 'defend', 'defend', 'atta

Even tough we have generated the data randomly, for the learning purposes, let's try to learn an HMM from this data. For this we have to construct the Baum-Welch algorithm from scratch. Below is the skeleton of the Baum-Welch learning algorithm.

## TASK 4: Complete the forward and backward probs functions in the Baum-Welch algorithm and try it with your formulated HMM.

In [25]:
import numpy as np


def baum_welch(observations, observations_vocab, n_hidden_states):
    """
    Baum-Welch algorithm for estimating the HMM parameters
    :param observations: observations
    :param observations_vocab: observations vocabulary
    :param n_hidden_states: number of hidden states to estimate
    :return: a, b (transition matrix and emission matrix)
    """

    def forward_probs(observations, observations_vocab, n_hidden_states, a_, b_) -> np.array:
        """
        forward pass to calculate alpha
        :param observations: observations
        :param observations_vocab: observation vocabulary
        :param n_hidden_states: number of hidden states
        :param a_: estimated alpha
        :param b_: estimated beta
        :return: refined alpha_
        """

        # Calculate the start probability
        a_start = 1 / n_hidden_states

        # Initialize alpha with zeros
        alpha_ = np.zeros((n_hidden_states, len(observations)), dtype=float)

        # Initialize the first column of alpha with the initial probabilities
        alpha_[:, 0] = a_start * b_[:, observations[0]]

        # Calculate alpha for the rest of the observations
        for t in range(1, len(observations)):
          # Iterate through each time step in the observations sequence, starting from the second time step (t = 1)
          for j in range(n_hidden_states):
            # Iterate through each hidden state

            # Calculate the sum of probabilities for each hidden state at time t-1, multiplied by the transition probabilities
            # from the previous hidden state to the current hidden state (j)
            proba_sum = sum(alpha_[:, t-1] * a_[:, j])

            # Update alpha with the calculated probability, multiplied by the emission probability
            # of observing the current observation (observations[t]) given the current hidden state (j)
            alpha_[j, t] = proba_sum * b_[j, observations[t]]
 
        # Return the alpha
        return alpha_

    def backward_probs(observations, observations_vocab, n_hidden_states, a_, b_) -> np.array:
        """
        backward pass to calculate alpha
        :param observations: observations
        :param observations_vocab: observation vocabulary
        :param n_hidden_states: number of hidden states
        :param a_: estimated alpha
        :param b_: estimated beta
        :return: refined beta_
        """

        # Initialize beta with zeros
        beta_ = np.zeros((n_hidden_states, len(observations)), dtype=float)

        # Set the last column of beta to 1 (Initialisation)
        beta_[:, -1:] = 1

        # Calculate beta for the rest of the observations
        for t in range(len(observations) -2, -1, -1):
          # The -2 : Start from the second to last observation, as the last one is already initialized
          # The -1 : Iterate until index 0 (inclusive)
          # The -1 : Iterate in reverse order (from the end to the beginning)
          for i in range(n_hidden_states):
            # Calculate the sum of probabilities for each hidden state
            proba_sum = sum(a_[i, :] * b_[:, observations[t + 1]] * beta_[:, t + 1])
            # Update beta with the calculated probability
            beta_[i, t] = proba_sum

        # We return the beta_
        return beta_

    def compute_gamma(alfa, beta, observations, vocab, n_samples, a_, b_) -> np.array:
        """

        :param alfa:
        :param beta:
        :param observations:
        :param vocab:
        :param n_samples:
        :param a_:
        :param b_:
        :return:
        """
        # gamma_prob = np.zeros(n_samples, len(observations))
        gamma_prob = np.multiply(alfa, beta) / sum(np.multiply(alfa, beta))
        return gamma_prob

    def compute_sigma(alfa, beta, observations, vocab, n_samples, a_, b_) -> np.array:
        """

        :param alfa:
        :param beta:
        :param observations:
        :param vocab:
        :param n_samples:
        :param a_:
        :param b_:
        :return:
        """
        sigma_prob = np.zeros((n_samples, len(observations) - 1, n_samples), dtype=float)
        denomenator = np.multiply(alfa, beta)
        for i in range(len(observations) - 1):
            for j in range(n_samples):
                for k in range(n_samples):
                    index_in_vocab = np.where(vocab == observations[i + 1])[0][0]
                    sigma_prob[j, i, k] = (alfa[j, i] * beta[k, i + 1] * a_[j, k] * b_[k, index_in_vocab]) / sum(
                        denomenator[:, j])
        return sigma_prob

    # initialize A ,B
    a = np.ones((n_hidden_states, n_hidden_states)) / n_hidden_states
    b = np.ones((n_hidden_states, len(observations_vocab))) / len(observations_vocab)
    for iter in tqdm(range(2000), position=0, leave=True):

        # E-step caclculating sigma and gamma
        alfa_prob = forward_probs(observations, observations_vocab, n_hidden_states, a, b)  #
        beta_prob = backward_probs(observations, observations_vocab, n_hidden_states, a, b)  # , beta_val
        gamma_prob = compute_gamma(alfa_prob, beta_prob, observations, observations_vocab, n_hidden_states, a, b)
        sigma_prob = compute_sigma(alfa_prob, beta_prob, observations, observations_vocab, n_hidden_states, a, b)

        # M-step caclculating A, B matrices
        a_model = np.zeros((n_hidden_states, n_hidden_states))
        for j in range(n_hidden_states):  # calculate A-model
            for i in range(n_hidden_states):
                for t in range(len(observations) - 1):
                    a_model[j, i] = a_model[j, i] + sigma_prob[j, t, i]
                normalize_a = [sigma_prob[j, t_current, i_current] for t_current in range(len(observations) - 1) for
                               i_current in range(n_hidden_states)]
                normalize_a = sum(normalize_a)
                if normalize_a == 0:
                    a_model[j, i] = 0
                else:
                    a_model[j, i] = a_model[j, i] / normalize_a

        b_model = np.zeros((n_hidden_states, len(observations_vocab)))

        for j in range(n_hidden_states):
            for i in range(len(observations_vocab)):
                indices = [idx for idx, val in enumerate(observations) if val == observations_vocab[i]]
                numerator_b = sum(gamma_prob[j, indices])
                denominator_b = sum(gamma_prob[j, :])
                if denominator_b == 0:
                    b_model[j, i] = 0
                else:
                    b_model[j, i] = numerator_b / denominator_b

        a = a_model
        b = b_model
    return a, b


import random

hidden_states = ['healthy', 'sick']
observable_states = ['sleeping', 'eating', 'pooping']
observable_map = {'sleeping': 0, 'eating': 1, 'pooping': 2}
observations = []
for i in range(100):
    observations.append(observable_map[random.choice(observable_states)])

A, B = baum_welch(observations=observations, observations_vocab=np.array(list(observable_map.values())),
                  n_hidden_states=2)

print(A)
print(B)


100%|██████████| 2000/2000 [00:10<00:00, 187.65it/s]

[[0.5 0.5]
 [0.5 0.5]]
[[0.28 0.35 0.37]
 [0.28 0.35 0.37]]


In [27]:
#TASK 4: Now try it with your HMM

import random

# Define the hidden states representing the player's skill level
hidden_states_own_HMM = ['beginner', 'expert']

# Define the observable states representing the actions performed by the player in the game
observable_states_own_HMM = ['attack', 'defend', 'use item']

# Define the map of the observable state
observable_map_own_HMM = {'attack': 0, 'defend': 1, 'use item': 2}

# Generate a list of 100 random observations from the observable states
observations_own_HMM = []
for i in range(100):
  observations_own_HMM.append(observable_map_own_HMM[random.choice(observable_states_own_HMM)])

print(observations_own_HMM)


A, B = baum_welch(observations=observations_own_HMM, observations_vocab=np.array(list(observable_map_own_HMM.values())),
                  n_hidden_states=2)

print(A)
print(B)

[0, 1, 1, 0, 2, 0, 2, 1, 2, 1, 2, 0, 2, 0, 1, 1, 1, 0, 0, 1, 1, 0, 2, 0, 0, 0, 1, 2, 0, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 2, 1, 1, 2, 1, 1, 0, 0, 1, 0, 0, 2, 2, 0, 2, 2, 2, 1, 1, 1, 0, 1, 0, 1, 2, 2, 1, 2, 2, 0, 0, 2, 0, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 2, 2, 2, 0, 1, 1, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 2]


100%|██████████| 2000/2000 [00:10<00:00, 184.02it/s]

[[0.5 0.5]
 [0.5 0.5]]
[[0.37 0.34 0.29]
 [0.37 0.34 0.29]]
